In [1]:
# Load libraries
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.svm import SVC
from scipy import stats
import seaborn as sns
import numpy as np

In [2]:
import requests # library to handle requests
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Folium installed
Libraries imported.


In [3]:
df = pd.read_csv("hawker_centre_modified_short.csv")
#Adding the dataset and turning it into a dataframe
df.head()

,name_of_centre,postal_code,location_of_centre,type_of_centre,owner,no_of_stalls,no_of_cooked_food_stalls,no_of_mkt_produce_stalls,Latitude,Longitude
0,Adam Road Food Centre,289876,"2, Adam Road",HC,Government,32,32,0,1.32926,103.81452
1,Amoy Street Food Centre,69111,"National Development Building, Annex B, Telok ...",HC,Government,135,134,1,1.28944,103.84998
2,Bedok Food Centre,469572,"1, Bedok Road",HC,Government,32,32,0,1.32659,103.95121
3,Beo Crescent Market,169982,"38A, Beo Crescent",MHC,Government,94,32,62,1.28864,103.82764
4,Berseh Food Centre,208877,"166, Jalan Besar",HC,Government,66,66,0,1.30578,103.85554


In [4]:
df.describe()

,postal_code,no_of_stalls,no_of_cooked_food_stalls,no_of_mkt_produce_stalls,Latitude,Longitude
count,33.000000,33.000000,33.000000,33.00000,33.000000,33.000000
mean,288985.272727,116.545455,58.545455,58.00000,1.310383,103.842926
std,174100.126842,84.297127,31.542720,72.50862,0.026512,0.046327
min,48940.000000,21.000000,16.000000,0.00000,1.280730,103.721626
25%,151115.000000,63.000000,32.000000,0.00000,1.289440,103.817009
50%,208877.000000,94.000000,51.000000,48.00000,1.305780,103.839730
75%,424768.000000,161.000000,83.000000,82.00000,1.320440,103.865840
max,618499.000000,365.000000,134.000000,302.00000,1.380500,103.951210


# Now I use the foursquare api

In [6]:
CLIENT_ID = 'TD5QDZZRT1HRKRDRY3ELQIJI4DNBT3X3IAKLLETWN4B1OYBN' # your Foursquare ID
CLIENT_SECRET = 'Q3H2QHFFJ5NYLYN5R1S2TG0NS4JR2IHY3BCCWFORPV534JR5' # your Foursquare Secret
VERSION = '20200420'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TD5QDZZRT1HRKRDRY3ELQIJI4DNBT3X3IAKLLETWN4B1OYBN
CLIENT_SECRET:Q3H2QHFFJ5NYLYN5R1S2TG0NS4JR2IHY3BCCWFORPV534JR5


In [7]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['CentreName', 
                  'Centre Latitude', 
                  'Centre Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
dp = getNearbyVenues(names=df['name_of_centre'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Adam Road Food Centre
Amoy Street Food Centre
Bedok Food Centre
Beo Crescent Market
Berseh Food Centre
Bukit Timah Market
Chomp Chomp Food Centre
Commonwealth Crescent Market
Dunman Food Centre
East Coast Lagoon Food Village
Geylang Serai Market
Golden Mile Food Centre
Holland Village Market & Food Centre
Kallang Estate Market
Market Street Food Centre
Newton Food Centre
North Bridge Road Market & Food Centre
Pasir Panjang Food Centre
Serangoon Garden Market
Sembawang Hill Food Centre
Taman Jurong Market & Food Centre
Tanglin Halt Market
Tiong Bahru Market
Zion Riverside Food Centre
ABC Brickworks Market & Food Centre
Albert Centre Market & Food Centre
Alexandra Village Food Centre
Blk 1 Jalan Kukoh
Blk 105 Hougang Ave 1
Blk 11 Telok Blangah Crescent
Blk 112 Jalan Bukit Merah
Blk 115 Bukit Merah View
Blk 117 Aljunied Ave 2


In [12]:
dp.head()

,CentreName,Centre Latitude,Centre Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adam Road Food Centre,1.32926,103.81452,Akane Restaurant,1.330851,103.815565,Japanese Restaurant
1,Adam Road Food Centre,1.32926,103.81452,Hollandse Club,1.329966,103.816648,Hotel
2,Adam Road Food Centre,1.32926,103.81452,Donguri,1.331121,103.815887,Japanese Restaurant
3,Adam Road Food Centre,1.32926,103.81452,7Adam,1.329441,103.813910,Modern European Restaurant
4,Adam Road Food Centre,1.32926,103.81452,Lounge,1.331060,103.815864,Lounge


In [16]:
dp.groupby('CentreName').count()

,Centre Latitude,Centre Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
CentreName,,,,,,
ABC Brickworks Market & Food Centre,24,24,24,24,24,24
Adam Road Food Centre,7,7,7,7,7,7
Albert Centre Market & Food Centre,58,58,58,58,58,58
Alexandra Village Food Centre,49,49,49,49,49,49
Amoy Street Food Centre,76,76,76,76,76,76
Bedok Food Centre,9,9,9,9,9,9
Beo Crescent Market,30,30,30,30,30,30
Berseh Food Centre,88,88,88,88,88,88
Blk 1 Jalan Kukoh,59,59,59,59,59,59


In [21]:
dp.shape

(1417, 191)